# Solutions to homework #3

In [1]:
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import requests
import numpy as np
import warnings

/usr/local/Caskroom/miniconda/base/envs/llm-zoomcamp/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
model_name = 'multi-qa-distilbert-cos-v1'
embedding_model = SentenceTransformer(model_name)

## Solution to question #1

In [4]:
user_question = "I just discovered the course. Can I still join it?"
vectors = embedding_model.encode(user_question)
print(f'Value of first vector (rounded to third decimal point: {round(vectors[0], 3):.3f}')

Value of first vector (rounded to third decimal point: 0.078


## Solution to question #2

In [5]:
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'

In [6]:
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [7]:
docs = [doc for doc in documents if doc['course'] in ['machine-learning-zoomcamp']]

In [8]:
embeddings = []
for doc in tqdm(docs):
    qa_text = f'''{doc['question']} {doc['text']}'''
    embeddings.append(embedding_model.encode(qa_text))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 375/375 [00:33<00:00, 11.17it/s]


In [9]:
X = np.array(embeddings)
print(f'Shape of the embeddings: {X.shape}')

Shape of the embeddings: (375, 768)


## Solution to question #3

In [10]:
v = vectors[0]
scores = X.dot(v)
max_score = scores.max()
print(f'Highest score from the first vector and the embeddings: {round(max_score, 4):.4f}')

Highest score from the first vector and the embeddings: 0.0127


In [11]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

In [12]:
# search_engine = VectorSearchEngine(documents=documents, embeddings=X)
# search_engine.search(v, num_results=5)